<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/stock/labelling_stock_selection2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('/content/nifty_50_weekly_data.csv')
# Convert the date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Sort the dataframe by ticker and date
df = df.sort_values(['ticker', 'date'])

# Group the dataframe by ticker
grouped = df.groupby('ticker')

# Function to calculate weekly returns
def calculate_weekly_return(group):
    # Shift the 'adj close' column by 1 day (1 month)
    group['next_week_close'] = group['adj close'].shift(-1)

    # Calculate the weekly return
    group['weekly_return'] = (group['next_week_close'] - group['adj close']) / group['adj close']

    return group

# Apply the function to each group (ticker)
df_with_returns = grouped.apply(calculate_weekly_return)

# Reset the index
df_with_returns = df_with_returns.reset_index(drop=True)

# Remove rows where we couldn't calculate the weekly return (last week of data for each stock)
df_with_returns = df_with_returns.dropna(subset=['weekly_return'])

# Display the first few rows of the result
print(df_with_returns[['date', 'ticker', 'adj close', 'next_week_close', 'weekly_return']].head())

# Calculate summary statistics of weekly returns for each stock
weekly_return_summary = df_with_returns.groupby('ticker')['weekly_return'].agg(['mean', 'median', 'std', 'min', 'max'])
print("\nWeekly Return Summary Statistics:")
print(weekly_return_summary)

# Find the stock with the highest average weekly return
best_performing_stock = weekly_return_summary['mean'].idxmax()
print(f"\nBest performing stock based on average weekly return: {best_performing_stock}")

# Save the results to a CSV file
df_with_returns.to_csv('stock_weekly_returns.csv', index=False)
print("\nResults saved to 'stock_weekly_returns.csv'")

        date       ticker   adj close  next_week_close  weekly_return
0 2020-05-03  ADANIENT.NS  141.048111       136.808716      -0.030056
1 2020-05-10  ADANIENT.NS  136.808716       140.699005       0.028436
2 2020-05-17  ADANIENT.NS  140.699005       137.756363      -0.020914
3 2020-05-24  ADANIENT.NS  137.756363       147.980835       0.074221
4 2020-05-31  ADANIENT.NS  147.980835       152.070633       0.027637

Weekly Return Summary Statistics:
                   mean    median       std       min       max
ticker                                                         
ADANIENT.NS    0.027453  0.023090  0.064544 -0.086110  0.305812
ADANIPORTS.NS  0.009769  0.012529  0.051029 -0.173147  0.118642
APOLLOHOSP.NS  0.009962  0.011657  0.049259 -0.140761  0.196830
ASIANPAINT.NS  0.005510  0.005277  0.041310 -0.122163  0.100867
AXISBANK.NS    0.006290  0.003213  0.048601 -0.141268  0.156090
BAJAJ-AUTO.NS  0.003910  0.004907  0.034759 -0.077860  0.142223
BAJAJFINSV.NS  0.011082  0.005355